# 📘 Topic Search & Dataset Normalization Pipeline

This notebook implements the preprocessing and normalization pipeline that feeds the semantic search engine and the learning-path recommender.  
It converts the raw Udemy Business marketplace export into a unified, analytics-ready dataset with:

- Cleaned and standardized text fields
- Normalized numeric metadata
- Consolidated semantic fields for embedding generation
- Consistent schema required for FAISS indexing and SBERT-based retrieval.

The final output of this notebook is `unified_courses_v1.csv`, which serves as the canonical dataset for the embedding pipeline and the downstream vector search system.

### 1. Dataset Loading & Initial Profiling
Load the raw marketplace dump and examine schema completeness, missing-value patterns, categorical cardinality, and numeric distributions.  
This step validates the integrity of the raw export before downstream normalization.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("raw_marketplace.csv")
print("Loaded dataset successfully.")

Loaded dataset successfully.


In [ ]:
df.head()

,courseid,course_title,instructor_title,category,subcategory,course_url,avg_rating_90d,num_rating_90d,num_rating_all_time,enrollments,hrs_of_content,duration_hours,course_mapping_keywords,rnk_enrollments,topic_title,topic_id
0,4918126,Principles of Secure Coding,Chris B Behrens,Development,Software Engineering,https://www.udemy.com/principles-of-secure-coding,4.48,1590,6299,878,3.3886,2.0,NaN,4880,Generic Programming,5930.0
1,4896784,No-Code and No-Math Machine Learning,Jones Granatyr,Data Science,Math & Statistics,https://www.udemy.com/no-code-no-math-machine-...,4.75,10,88,1455,5.1229,0.0,Machine Learning,310,Machine Learning,6594.0
2,4896268,How to give and receive feedback effectively,Jamal Lazaar,Communication,Fundamentals of Communication,https://www.udemy.com/give-and-receive-feedback,4.64,286,1490,4973,1.1391,0.0,NaN,4153,Interpersonal Feedback,89930.0
3,4879342,Prepare for the Salesforce Certified Associate...,"Emily Call, MBA",IT Operations,IT Certifications,https://www.udemy.com/prepare-for-the-salesfor...,4.47,223,2515,1092,3.9997,1.0,NaN,4840,Salesforce Certified Associate,157870.0
4,4878666,Google Cloud Machine Learning Engineer Certifi...,Dan Sullivan,IT Operations,IT Certifications,https://www.udemy.com/google-cloud-machine-lea...,4.29,121,664,1716,5.2511,0.0,Machine Learning,306,Google Cloud Professional Machine Learning Eng...,156700.0


In [ ]:
df.info()
df.describe(include='all').T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26232 entries, 0 to 26231
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   courseid                 26232 non-null  int64  
 1   course_title             26232 non-null  object 
 2   instructor_title         26232 non-null  object 
 3   category                 26232 non-null  object 
 4   subcategory              26232 non-null  object 
 5   course_url               26232 non-null  object 
 6   avg_rating_90d           26232 non-null  float64
 7   num_rating_90d           26232 non-null  int64  
 8   num_rating_all_time      26232 non-null  int64  
 9   enrollments              26232 non-null  int64  
 10  hrs_of_content           25938 non-null  float64
 11  duration_hours           26229 non-null  float64
 12  course_mapping_keywords  7321 non-null   object 
 13  rnk_enrollments          26232 non-null  int64  
 14  topic_title           

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
courseid,26232.0,NaN,NaN,NaN,1689901.771005,1260061.053054,8139.0,735420.0,1320528.0,2490968.0,4918126.0
course_title,26232,7804,Java Multithreading,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
instructor_title,26232,2920,Simon Sez IT,235,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category,26232,32,Development,6814,NaN,NaN,NaN,NaN,NaN,NaN,NaN
subcategory,26232,195,Web Development,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN
course_url,26232,7510,https://www.udemy.com/java-multithreading,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
avg_rating_90d,26232.0,NaN,NaN,NaN,4.527636,0.127014,4.25,4.45,4.52,4.61,5.0
num_rating_90d,26232.0,NaN,NaN,NaN,248.452005,819.029425,9.0,25.0,69.0,189.0,29558.0
num_rating_all_time,26232.0,NaN,NaN,NaN,5735.749733,16164.598722,9.0,658.0,1656.0,4607.25,499646.0
enrollments,26232.0,NaN,NaN,NaN,110369.533089,237066.313468,30.0,12628.25,39701.0,107847.5,5334056.0


### 2. Text-Field Standardization
Normalize all text fields to ensure consistent input to the embedding model:
- cast to string,
- trim whitespace,
- replace nulls with empty strings,
preventing tokenization errors and inconsistent semantic representations.

In [ ]:
text_cols = [
    'courseid', 'course_title', 'instructor_title',
    'category', 'subcategory', 'course_url',
    'course_mapping_keywords', 'topic_title'
]
for col in text_cols:
    df[col] = df[col].astype(str).str.strip().fillna("")

### 3. Numeric Field Normalization
Unify numeric schema by standardizing column names and coercing inconsistent types.  
`duration_hours` and `topic_id` are cleaned to ensure stable downstream use in ranking logic and filtering operations.

In [ ]:
df = df.drop(columns=['duration_hours'])
df = df.rename(columns={'hrs_of_content': 'duration_hours'})
df[['duration_hours']].describe()

,duration_hours
count,25938.000000
mean,12.537945
std,79.418866
min,0.243700
25%,2.769100
50%,5.510000
75%,11.664575
max,2300.931400


In [ ]:
df['topic_id'] = pd.to_numeric(df['topic_id'], errors='coerce')
df['topic_id'] = df['topic_id'].fillna(-1)
df['topic_id'] = df['topic_id'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26232 entries, 0 to 26231
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   courseid                 26232 non-null  object 
 1   course_title             26232 non-null  object 
 2   instructor_title         26232 non-null  object 
 3   category                 26232 non-null  object 
 4   subcategory              26232 non-null  object 
 5   course_url               26232 non-null  object 
 6   avg_rating_90d           26232 non-null  float64
 7   num_rating_90d           26232 non-null  int64  
 8   num_rating_all_time      26232 non-null  int64  
 9   enrollments              26232 non-null  int64  
 10  duration_hours           25938 non-null  float64
 11  course_mapping_keywords  26232 non-null  object 
 12  rnk_enrollments          26232 non-null  int64  
 13  topic_title              26232 non-null  object 
 14  topic_id              

### 4. Embedding-Text Construction
Create a consolidated semantic field combining course title, category, subcategory, topic, and curated keywords.  
This engineered field is the input to SBERT/GTE for vector encoding and significantly improves retrieval quality.

In [ ]:
def make_embedding_text(row):
    parts = []
    parts.append(str(row['course_title']))
    parts.append(str(row['category']))
    parts.append(str(row['subcategory']))

    if row['topic_title'].strip() != "":
        parts.append(str(row['topic_title']))
    if row['course_mapping_keywords'].strip() != "":
        parts.append(str(row['course_mapping_keywords']))
    return " | ".join(parts)

df['embedding_text'] = df.apply(make_embedding_text, axis=1)
df[['course_title','embedding_text']].head(10)

,course_title,embedding_text
0,Principles of Secure Coding,Principles of Secure Coding | Development | So...
1,No-Code and No-Math Machine Learning,No-Code and No-Math Machine Learning | Data Sc...
2,How to give and receive feedback effectively,How to give and receive feedback effectively |...
3,Prepare for the Salesforce Certified Associate...,Prepare for the Salesforce Certified Associate...
4,Google Cloud Machine Learning Engineer Certifi...,Google Cloud Machine Learning Engineer Certifi...
5,Google Cloud Machine Learning Engineer Certifi...,Google Cloud Machine Learning Engineer Certifi...
6,Thriving in a Hybrid Environment: Tactics for ...,Thriving in a Hybrid Environment: Tactics for ...
7,Thriving in a Hybrid Environment: Tactics for ...,Thriving in a Hybrid Environment: Tactics for ...
8,Thriving in a Hybrid Environment: Tactics for ...,Thriving in a Hybrid Environment: Tactics for ...
9,Artificial Intelligence AI Marketing to Grow y...,Artificial Intelligence AI Marketing to Grow y...


### 5. Export Clean Unified Dataset
Export the canonical dataset used by:
- the SBERT embedding pipeline,
- FAISS vector index builder,
- semantic search engine,
- learning-path generator.

This file represents the fully normalized and embedding-ready version of the Udemy catalog.

In [ ]:
df.to_csv("unified_courses_v1.csv", index=False)

In [ ]:
print("Number of categories:", df['category'].nunique())
print("Number of subcategories:", df['subcategory'].nunique())
print("Number of topics:", df['topic_title'].nunique())

print("\nSample categories:")
print(df['category'].value_counts().head(10))

print("\nSample topics:")
print(df['topic_title'].value_counts().head(15))

Number of categories: 32
Number of subcategories: 195
Number of topics: 2179

Sample categories:
category
Development                6814
IT Operations              3319
Data Science               1989
Personal Development       1958
Leadership & Management    1298
Finance & Accounting       1081
Marketing                   934
Productivity                795
Cloud Computing             774
Design Tools                772
Name: count, dtype: int64

Sample topics:
topic_title
Python                       340
Java                         274
Excel                        242
Leadership                   188
JavaScript                   180
SQL                          137
Project Management           136
Agile                        134
Amazon AWS                   130
Data Analysis                123
Business Analysis            121
C# (programming language)    111
Presentation Skills          109
Sales Skills                 106
Management Skills            105
Name: count, dtype: int64

In [ ]:
def find_topics(keyword, top_n=20):
    keyword = keyword.lower()
    mask = df['topic_title'].str.lower().str.contains(keyword)
    return df.loc[mask, 'topic_title'].value_counts().head(top_n)
# Example:
find_topics("python")

topic_title
Python                                            340
Certified Entry-Level Python Programmer (PCEP)     10
Python Network Programming                          7
MicroPython                                         4
Python GUI                                          2
Python Hacking                                      2
Python Scripting                                    2
Name: count, dtype: int64

In [ ]:
def search_by_title(keyword, top_n=20):
    keyword = keyword.lower()
    mask = df['course_title'].str.lower().str.contains(keyword)
    results = df.loc[mask, [
        'courseid', 'course_title', 'category', 'subcategory',
        'avg_rating_90d', 'num_rating_all_time', 'enrollments',
        'duration_hours'
    ]].copy()

    # sort by rating + review num
    results = results.sort_values(
        by=['avg_rating_90d', 'num_rating_all_time', 'enrollments'],
        ascending=[False, False, False]
    )

    return results.head(top_n)

# test:
search_by_title("python").head(10)


,courseid,course_title,category,subcategory,avg_rating_90d,num_rating_all_time,enrollments,duration_hours
18318,3663114,Mastering 4 critical SKILLS using Python,IT Operations,IT Certifications,4.93,890,8987,28.5329
18319,3663114,Mastering 4 critical SKILLS using Python,IT Operations,IT Certifications,4.93,890,8987,28.5329
1500,3663114,Mastering 4 critical SKILLS using Python,IT Operations,IT Certifications,4.91,900,8930,28.5329
1501,3663114,Mastering 4 critical SKILLS using Python,IT Operations,IT Certifications,4.91,900,8930,28.5329
21765,1647776,"Python 3: Deep Dive (Part 2 - Iterators, Gener...",Development,Programming Languages,4.85,2526,87724,36.1577
1467,3687702,Short Programming intro in Python and C++,Development,Programming Languages,4.85,296,5423,9.2061
18289,3687702,Short Programming intro in Python and C++,Development,Programming Languages,4.85,296,5417,9.2061
3542,2358038,Learn 100+ Python Pattern Printing Programs In...,Development,Programming Languages,4.85,179,16144,46.4227
20361,2358038,Learn 100+ Python Pattern Printing Programs In...,Development,Programming Languages,4.85,178,16166,46.4227
4921,1647776,"Python 3: Deep Dive (Part 2 - Iterators, Gener...",Development,Programming Languages,4.84,2560,87370,36.1577


In [ ]:
def search_by_topic(topic_name, top_n=20):
    # exact match, case-insensitive
    topic_name_lower = topic_name.lower()
    mask = df['topic_title'].str.lower() == topic_name_lower

    topic_df = df.loc[mask, [
        'courseid', 'course_title', 'category', 'subcategory',
        'avg_rating_90d', 'num_rating_all_time', 'enrollments',
        'duration_hours'
    ]].copy()

    if topic_df.empty:
        print("No course found for topic:", topic_name)
        return topic_df

    topic_df = topic_df.sort_values(
        by=['avg_rating_90d', 'num_rating_all_time', 'enrollments'],
        ascending=[False, False, False]
    )

    return topic_df.head(top_n)

In [ ]:
search_by_topic("Machine Learning")

,courseid,course_title,category,subcategory,avg_rating_90d,num_rating_all_time,enrollments,duration_hours
17977,3914960,Machine Learning for Data Analysis: Classifica...,Data Science,Machine Learning,4.87,464,12169,2.5146
1657,3565567,Machine Learning with Imbalanced Data,Data Science,Data Visualization,4.85,717,21900,9.1628
16844,4896784,No-Code and No-Math Machine Learning,Data Science,Math & Statistics,4.85,86,1445,5.1229
1147,3914960,Machine Learning for Data Analysis: Classifica...,Data Science,Machine Learning,4.80,467,12121,2.5146
5200,1548436,Feature Selection for Machine Learning,Data Science,Machine Learning,4.75,2143,62937,5.8295
5234,1533128,Clustering & Classification With Machine Learn...,Data Science,Machine Learning,4.75,367,6029,6.0704
6380,1221664,Applied Machine Learning in R,Development,DevOps,4.75,258,1229,8.1795
23259,1221664,Applied Machine Learning in R,Development,DevOps,4.75,256,1223,8.1795
1,4896784,No-Code and No-Math Machine Learning,Data Science,Math & Statistics,4.75,88,1455,5.1229
4263,1955654,Machine Learning with Javascript,Data Science,Machine Learning,4.74,3230,71393,17.6790


In [ ]:
find_topics("machine")

topic_title
Machine Learning                                       99
AWS Certified Machine Learning - Specialty             14
Azure Virtual Machines                                  8
Azure Machine Learning                                  5
Google Cloud Professional Machine Learning Engineer     4
Java Virtual Machine                                    3
Supervised Machine Learning                             2
Unsupervised Machine Learning                           2
AutoML Automated Machine Learning                       2
World Machine                                           2
Name: count, dtype: int64

In [ ]:
def print_top_courses_for_topic(topic_name, top_n=10):
    results = search_by_topic(topic_name, top_n=top_n)
    if results.empty:
        return
    
    display_cols = [
        'course_title', 'avg_rating_90d',
        'num_rating_all_time', 'enrollments', 'duration_hours'
    ]
    print(f"Top {top_n} courses for topic: {topic_name}\n")
    print(results[display_cols])

# test:
print_top_courses_for_topic("Machine Learning", top_n=5)

Top 5 courses for topic: Machine Learning

                                            course_title  avg_rating_90d  \
17977  Machine Learning for Data Analysis: Classifica...            4.87   
1657               Machine Learning with Imbalanced Data            4.85   
16844               No-Code and No-Math Machine Learning            4.85   
1147   Machine Learning for Data Analysis: Classifica...            4.80   
5200              Feature Selection for Machine Learning            4.75   

       num_rating_all_time  enrollments  duration_hours  
17977                  464        12169          2.5146  
1657                   717        21900          9.1628  
16844                   86         1445          5.1229  
1147                   467        12121          2.5146  
5200                  2143        62937          5.8295  
